<a href="https://colab.research.google.com/github/kamilkaminski01/PRiR/blob/master/src/lab5/prir_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Programowanie Równoległe i Rozproszone**
# Kamil Kamiński LAB gr.1

# **Laboratorium nr 5**

**Zadanie 1** - Zaimplementuj w środowisku MPI obliczanie równoległe liczby PI z wzoru Leibniz-a.

Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave, indeksy wyrazów szeregu poczatek_local i koniec_local, z których należy obliczyć wyniki cząstkowe
- zbiera wyniki cząstkowe z procesów i wyświetla wynik przybliżenia PI

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <stdio.h>
#include <mpi.h>
#include <stdlib.h>
#include <math.h>

int main(int argc, char **argv)
{
    int rank;
    int num_proc;
    int tag = 1;
    float wynik = 0;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &num_proc);
    MPI_Status status;

    double start = MPI_Wtime();
    if(rank!=0)
    {
        MPI_Recv(&wynik, 1, MPI_FLOAT, rank-1, tag, MPI_COMM_WORLD, &status);
    }

    // Wzor Leibniza
    wynik = wynik + powf(-1, rank) / (2 * (rank+1) - 1) * 4;

    if(rank != num_proc-1)
    {
        MPI_Send(&wynik, 1, MPI_FLOAT, rank+1, tag, MPI_COMM_WORLD);
    }
    else
    {
        double end = MPI_Wtime();
        printf("Przyblizenie PI = %f\n", wynik);
        printf("Czas obliczen %f",end - start);
    }
    MPI_Finalize();
    return 0;
}

EOF
mpicc pi-mpi.c -lm && mpirun -n 22 --allow-run-as-root a.out

Przyblizenie PI = 3.096161
Czas obliczen 0.010571

**Zadanie 2** - Zaimplementuj w środowisku MPI wyznaczenie numerycznej wartość całki y=f(x) (postać funkcji wybierasz sam) w przedziale <a,b> przy pomocy N trapezów.

Proces 0, to proces Root, który zarządza obliczeniami:
- wysyła do procesów roboczych Slave początek a_local i koniec b_local lokalnego przedziału całkowania dla danego procesu oraz liczbę N_local trapezów, z których należy policzyć całkę
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <stdio.h>
#include <mpi.h>
#include <math.h>
#include <stdlib.h>

double wzorfunkcji(double x)
{
  return pow(x, 2);
}

double wys(double a, double b, double N)
{
  return (b - a) / N;
}

int main(int argc,char **argv)
{
  int rank;
  int num_proc;
  int tag = 5;
  MPI_Status status;

  MPI_Init(&argc,&argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &num_proc);

  // Metoda Trapezów
  double a = 2;
  double b = 4;
  double xi = 0;
  double suma = 0;

  double start = MPI_Wtime();

  if(rank == num_proc - 1)
  { 
    xi += (wzorfunkcji(a + (((double) rank / (double) num_proc) * (b - a))));
    MPI_Send(&xi,1,MPI_DOUBLE,rank-1,tag,MPI_COMM_WORLD);

  }
  else if(rank != 0 && rank < num_proc - 1)
  { 
    MPI_Recv(&xi,1,MPI_DOUBLE,rank+1,tag,MPI_COMM_WORLD,&status);
    xi += (wzorfunkcji(a + (((double) rank / (double) num_proc) * (b - a))));           
    MPI_Send(&xi,1,MPI_DOUBLE,rank-1,tag,MPI_COMM_WORLD);
  }
  else
  {
    MPI_Recv(&xi,1,MPI_DOUBLE,rank+1,tag,MPI_COMM_WORLD,&status);
    suma = wys(a,b,num_proc) * (wzorfunkcji(a) / 2 + xi + wzorfunkcji(b)/2);
    double end = MPI_Wtime();
    printf("Wynik całki numerycznej metodą trapezów = %f \n", suma);
    printf("Czas obliczen %f",end - start);
  }

  MPI_Finalize();
  return 0;
}

EOF
mpicc pi-mpi.c -lm && mpirun -n 4 --allow-run-as-root a.out

Wynik całki numerycznej metodą trapezów = 18.750000 
Czas obliczen 0.000414

**Zadanie 3** - Zaimplementuj program mnożenia macierzy A przez wektor B

Proces 0, to proces Root, który zarządza obliczeniami:
- inicjuje wartości macierzy A i wektora B
- wysyła do procesów roboczych Slave fragmenty macierzy A i wektora B 
- zbiera wyniki cząstkowe z procesów, i wyświetla wynik całki

In [2]:
%%sh
cat > pi-mpi.c << EOF

#include <stdlib.h> 
#include <stdio.h> 
#include <mpi.h> 
#include <time.h> 
#include <sys/time.h>

#define N 4
MPI_Status status;
double A[N][N];
double B[N],C[N];

int main(int argc, char **argv) 
{
  int processCount, processId, slaveTaskCount, source, dest, rows, offset;

  struct timeval start, stop;

  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &processId);
  MPI_Comm_size(MPI_COMM_WORLD, &processCount);


  slaveTaskCount = processCount - 1;

  if (processId == 0) 
  {
    double start = MPI_Wtime();
    
    srand(time(NULL));
    for (int i = 0; i<N; i++) 
    {
      for (int j = 0; j<N; j++) 
      {
        A[i][j]= rand()%10;
      }
      B[i]= rand()%10;
    }
    printf("\n Mnozenie macierzy przez wektor za pomoca MPI \n");
    
    printf("\nMacierz A\n\n");
    for (int i = 0; i<N; i++)
    {
      for (int j = 0; j<N; j++)
      {
        printf("%.0f\t", A[i][j]);
      }
      printf("\n");
    }
    printf("\nWektor B\n\n");
    for (int i = 0; i<N; i++)
    {
      printf("%.0f\t", B[i]);
      printf("\n");
    }

    rows = N/slaveTaskCount;
    offset = 0;

    for (dest=1; dest <= slaveTaskCount; dest++)
    {
      MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
      MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
      MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);
      MPI_Send(&B, N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
      offset = offset + rows;
    }

    for (int i = 1; i <= slaveTaskCount; i++)
    {
      source = i;
      MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
      MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
      MPI_Recv(&C[offset], rows*N, MPI_DOUBLE, source, 2, MPI_COMM_WORLD, &status);
    }

    printf("\nWynikowy wektor C:\n\n");
    for (int i = 0; i<N; i++) 
    {
      printf("%.0f\t", C[i]);

    }
    printf ("\n");

    double end = MPI_Wtime();
    printf ("\n");
    printf("Czas obliczen: %f",end - start);
  }
  // Kod do wykonania przez procesy Slave
  if (processId > 0) 
  {
    source = 0;
    MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
    MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
    MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
    MPI_Recv(&B, N*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
    for (int i = 0; i<rows; i++)
    {
      for (int j = 0; j<N; j++)
      {
        C[i] = C[i] + A[i][j] * B[j];
      }
    }

    MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
    MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);
    MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
  }
  MPI_Finalize();
}

EOF
mpicc pi-mpi.c && mpirun -n 4 --allow-run-as-root a.out


 Mnozenie macierzy przez wektor za pomoca MPI 

Macierz A

8	6	6	9	
7	1	4	6	
9	4	1	0	
2	3	4	7	

Wektor B

7	
4	
4	
4	

Wynikowy wektor C:

140	93	83	0	

Czas obliczen: 0.000528